In [28]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

%matplotlib inline
import shutil
import brtdevkit.util.s3 as brt_s3
import requests
import skimage
import skimage.transform as skt
from tqdm import tqdm

import os
import math
#import xmltodict
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Devkit
import brtdevkit
from brtdevkit.core.db import DBConnector
from brtdevkit.core.db.db_filters import * 

from collections import defaultdict, namedtuple

sns.set_style('whitegrid')
sns.set_context('notebook')

from mpl_toolkits.axes_grid1 import ImageGrid
from pathlib import Path
from tqdm import tqdm

from utils import *

In [9]:
def plot_bounding_boxes(ax, boxes, edgecolor=None):
    for box in sorted(boxes, key=lambda box: box.area_sq_px, reverse=True):
        edgecolor_ = edgecolor
        if edgecolor is None:
            edgecolor_ = 'r' if box.name == 'weed' else 'g'
        ax.add_patch(patches.Rectangle((box.xmin, box.ymin), box.width_px, box.height_px,
                                        fill=True,
                                        facecolor='none',
                                        edgecolor=edgecolor_,
                                        alpha=1,
                                        lw=2,
                                        antialiased=False))

In [14]:
# Start date of the data we care about
start_date =datetime(2020, 1, 20)

# End date of the data we care about
end_date =datetime(2020, 7, 9)

crop_name = 'SOYBEANS'

model_name = '20200528_1_soybeans'

out_dir = Path('analysis')


In [15]:
db = DBConnector()

img_filters = [
    {
        'project_name': 'shasta',
        "crop_name": 'SOYBEANS',
        'artifacts': {"$elemMatch": {'s3_bucket': {"$exists": True}, 's3_key': {"$exists": True}}},
        "annotations": {"$elemMatch": {"is_active_version": True, "state": "ok", "kind": {"$nin": ["ndvi_mask", "machine"]}, "style": 'pixelwise', 's3_bucket': {"$exists": True}, 's3_key': {"$exists": True},  'label_map': {'$in': [{'1': 'crop', '2': 'weed'}, {'1': 'weed', '2': 'crop'}]}}},
    }, 
    DatetimeFilter(key='collected_on', start=start_date, end=end_date)
]

df = db.get_documents_df('image', img_filters, limit=None)
print("I have a dataframe of length: ", len(df))

I have a dataframe of length:  27828


In [43]:
# Filter by farm

bbox = pd.read_csv('output_ben_cline_3.csv')
bbox['_id']=bbox['dp_id']
bbox.set_index('_id')
df_filtered = df[df['operating_field_name']=='behind freddy']
df_filtered.drop(['_id'], axis=1,inplace=True)
print(len(df_filtered))

dm = df_filtered.merge(bbox, on='_id')
print(len(dm))
dm.columns

470
0


Index(['_id', 'is_deleted', 'collected_on', 'gnss', 'annotations', 'artifacts',
       'has_human_annotation', 'uuid', 'angle_to_row', 'camera_exposure_algo',
       'camera_exposure_time', 'camera_focal_length', 'camera_fstop',
       'camera_gain', 'camera_height', 'camera_manufacturer', 'camera_ppi',
       'camera_revision', 'camera_sensor_name', 'camera_sensor_type',
       'camera_serial', 'cloud_cover', 'crop_damage', 'crop_health',
       'crop_height', 'crop_name', 'crop_residue', 'experiment_id', 'farm',
       'furrow_depth_inches', 'gps_course', 'gps_quality', 'gps_speed',
       'group_id', 'grower', 'icp_version', 'isp_version', 'latitude',
       'longitude', 'notes', 'operating_field_name', 'operator_name',
       'project_name', 'raw_bit_depth', 'robot_name', 'row_spacing',
       'soil_color', 'tillage_practice', 'type', 'weed_pressure', 'weeds',
       'artifacts_content_hash', 'no_crop_metadata', 'dp_id', 'mistakes'],
      dtype='object')

In [29]:
# Acts as a temp folder, deletes previous existing data

if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
out_dir.mkdir(parents=False, exist_ok=True)

model_dir = out_dir / Path('ShastaModel')
model_dir.mkdir(parents=False, exist_ok=True)

img_dir = out_dir / Path('Images')
img_dir.mkdir(parents=False, exist_ok=True)

preds_dir = out_dir / Path('Preds')
preds_dir.mkdir(parents=False, exist_ok=True)

In [31]:
def download_assets(directory, df):
    """
    Downloads s3 assets of a given list
    Args:
        directory (Path): the path that the assets will be saved to
        assets (iter(dict)): an iterable containing metadata objects that have s3 assets.
            Will usually be a dataframe column (ex: `image_df['artifacts'].explode()`)
    """
    s3_client = brt_s3.S3()

    def _download(asset, filepath):
        filepath.parent.mkdir(parents=True, exist_ok=True)
        return s3_client.download_file(asset['s3_bucket'], asset['s3_key'], str(filepath))

    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        datapoint_id = str(row['dp_id'])
        try:
            annotation = [a for a in row['annotations'] if a['is_active_version'] and a['kind'] != "ndvi_mask" and a['style'] == 'pixelwise'][0]
        except:
            annotation = [a for a in row['annotations'] if a['kind'] in ['f8', 'labelbox', 'brt', 'dataloop']][0]

        artifact = [a for a in row['artifacts'] if a['kind'] == 'nrg'][0]
        _download(artifact, directory / f'{datapoint_id}_nrg.png')
        _download(annotation, directory / f'{datapoint_id}_ann.png')
          

download_assets(img_dir, df_filtered)

  0%|          | 0/10 [00:00<?, ?it/s]


KeyError: 'annotations'